In [11]:
from dotenv import load_dotenv
from pydantic_ai import Agent, NativeOutput
from rich import print as rich_print

load_dotenv()

True

In [12]:
import ollama
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.ollama import OllamaProvider

OLLAMA_MODEL_NAME = "gpt-oss:20b"
OLLAMA_BASE_URL = "http://localhost:11434"

ollama.pull(OLLAMA_MODEL_NAME)

model = OpenAIChatModel(
    model_name=OLLAMA_MODEL_NAME,
    provider=OllamaProvider(base_url=f"{OLLAMA_BASE_URL}/v1", api_key="ollama"),
)

In [13]:
naive_agent = Agent(model)

In [36]:
result = await naive_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

**First 10 Presidents of the United States**

| # | President | Term |
|---|-----------|------|
| 1 | George Washington | 1789‑1797 |
| 2 | John Adams | 1797‑1801 |
| 3 | Thomas Jefferson | 1801‑1809 |
| 4 | James Madison | 1809‑1817 |
| 5 | James Monroe | 1817‑1825 |
| 6 | John Quincy Adams | 1825‑1829 |
| 7 | Andrew Jackson | 1829‑1837 |
| 8 | Martin Van Buren | 1837‑1841 |
| 9 | William Henry Harrison | 1841 |
|10 | John Tyler | 1841‑1845 |

These are the first ten individuals to hold the presidency of the United States.

RunUsage(input_tokens=81, output_tokens=283, requests=1)

In [37]:
from pydantic_ai import ModelSettings


result = await naive_agent.run('give me the names of the first 10 presidents of the united states', model_settings=ModelSettings(temperature=1.4))
rich_print(result.output)
rich_print(result.usage())

**The first 10 Presidents of the United States:**

1. George Washington  
2. John Adams  
3. Thomas Jefferson  
4. James Madison  
5. James Monroe  
6. John Quincy Adams  
7. Andrew Jackson  
8. Martin Van Buren  
9. William Henry Harrison  
10. John Tyler

RunUsage(input_tokens=81, output_tokens=214, requests=1)

In [16]:
from typing import List
from pydantic import BaseModel


class PresidentialTerm(BaseModel):
    name: str
    term_start_year: int
    term_end_year: int


class Presidents(BaseModel):
    presidents: List[PresidentialTerm]


# Add output type

This one will fail, due to not telling the model the type we expect
while also requiring the output pass validation

In [17]:
default_typed_output_agent = Agent[None, Presidents](
    model,
    output_type=Presidents,
)

In [18]:
try:
    result = await default_typed_output_agent.run('give me the names of the first 10 presidents of the united states')
except Exception as e:
    rich_print(e)


Exceeded maximum retries (1) for output validation

# NativeOutput

Specify to the model that output should be in JSON, of a certain structure

In [19]:
native_output_agent = Agent[None, Presidents](
    model,
    output_type=NativeOutput(
        Presidents,
        name='Presidents',
        description='A list of presidential terms',
    ),
)

In [20]:
result = await native_output_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())

Presidents(
    presidents=[
        PresidentialTerm(name='George Washington', term_start_year=1789, term_end_year=1797),
        PresidentialTerm(name='John Adams', term_start_year=1797, term_end_year=1801),
        PresidentialTerm(name='Thomas Jefferson', term_start_year=1801, term_end_year=1809),
        PresidentialTerm(name='James Madison', term_start_year=1809, term_end_year=1817),
        PresidentialTerm(name='James Monroe', term_start_year=1817, term_end_year=1825),
        PresidentialTerm(name='John Quincy Adams', term_start_year=1825, term_end_year=1829),
        PresidentialTerm(name='Andrew Jackson', term_start_year=1829, term_end_year=1837),
        PresidentialTerm(name='Martin Van Buren', term_start_year=1837, term_end_year=1841),
        PresidentialTerm(name='William Henry Harrison', term_start_year=1841, term_end_year=1841),
        PresidentialTerm(name='John Tyler', term_start_year=1841, term_end_year=1845)
    ]
)

RunUsage(input_tokens=136, output_tokens=201, requests=1)

In [21]:
rich_print(result.all_messages())

[
    ModelRequest(parts=[UserPromptPart(content='give me the names of the first 10 presidents of the united states',
timestamp=datetime.datetime(2025, 11, 5, 16, 2, 7, 853626, tzinfo=datetime.timezone.utc))]),
    ModelResponse(parts=[ThinkingPart(content='User asks: "give me the names of the first 10 presidents of the 
United States". Likely expects list. We can give names in order. Could provide with numbering. Straight answer. 
Let\'s provide list. It\'s simple.', id='reasoning', provider_name='ollama'), 
TextPart(content='{"presidents":[{"name":"George 
Washington","term_start_year":1789,"term_end_year":1797},{"name":"John 
Adams","term_start_year":1797,"term_end_year":1801},{"name":"Thomas 
Jefferson","term_start_year":1801,"term_end_year":1809},{"name":"James 
Madison","term_start_year":1809,"term_end_year":1817},{"name":"James 
Monroe","term_start_year":1817,"term_end_year":1825},{"name":"John Quincy 
Adams","term_start_year":1825,"term_end_year":1829},{"name":"Andrew 
Jackson","term_start_year":1829,"term_end_year":1837},{"name":"Martin Van 
Buren","term_start_year":1837,"term_end_year":1841},{"name":"William Henry 
Harrison","term_start_year":1841,"term_end_year":1841},{"name":"John 
Tyler","term_start_year":1841,"term_end_year":1845}]}')], usage=RequestUsage(input_tokens=136, output_tokens=201), 
model_name='gpt-oss:20b', timestamp=datetime.datetime(2025, 11, 5, 16, 2, 15, tzinfo=TzInfo(0)), 
provider_name='ollama', provider_details={'finish_reason': 'stop'}, provider_response_id='chatcmpl-285', 
finish_reason='stop')
]

In [22]:
result = await native_output_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)

Presidents(
    presidents=[
        PresidentialTerm(name='George Washington', term_start_year=1789, term_end_year=1797),
        PresidentialTerm(name='John Adams', term_start_year=1797, term_end_year=1801),
        PresidentialTerm(name='Thomas Jefferson', term_start_year=1801, term_end_year=1809),
        PresidentialTerm(name='James Madison', term_start_year=1809, term_end_year=1817),
        PresidentialTerm(name='James Monroe', term_start_year=1817, term_end_year=1825),
        PresidentialTerm(name='John Quincy Adams', term_start_year=1825, term_end_year=1829),
        PresidentialTerm(name='Andrew Jackson', term_start_year=1829, term_end_year=1837),
        PresidentialTerm(name='Martin Van Buren', term_start_year=1837, term_end_year=1841),
        PresidentialTerm(name='William Henry Harrison', term_start_year=1841, term_end_year=1841),
        PresidentialTerm(name='John Tyler', term_start_year=1841, term_end_year=1845)
    ]
)

# PromptedOutput

prompt in text that the output should be in a certain JSON structure

In [23]:
from pydantic_ai import PromptedOutput

prompted_output_agent = Agent[None, Presidents](
    model,
    output_type=PromptedOutput(
        Presidents,
        name='Presidents',
        description='A list of presidential terms',
    ),
)

In [24]:
result = await prompted_output_agent.run('give me the names of the first 10 presidents of the united states')
rich_print(result.output)
rich_print(result.usage())
rich_print(result.all_messages())

Presidents(
    presidents=[
        PresidentialTerm(name='George Washington', term_start_year=1789, term_end_year=1797),
        PresidentialTerm(name='John Adams', term_start_year=1797, term_end_year=1801),
        PresidentialTerm(name='Thomas Jefferson', term_start_year=1801, term_end_year=1809),
        PresidentialTerm(name='James Madison', term_start_year=1809, term_end_year=1817),
        PresidentialTerm(name='James Monroe', term_start_year=1817, term_end_year=1825),
        PresidentialTerm(name='John Quincy Adams', term_start_year=1825, term_end_year=1829),
        PresidentialTerm(name='Andrew Jackson', term_start_year=1829, term_end_year=1837),
        PresidentialTerm(name='Martin Van Buren', term_start_year=1837, term_end_year=1841),
        PresidentialTerm(name='William Henry Harrison', term_start_year=1841, term_end_year=1841),
        PresidentialTerm(name='John Tyler', term_start_year=1841, term_end_year=1845)
    ]
)

RunUsage(input_tokens=437, output_tokens=201, requests=1)

[
    ModelRequest(parts=[UserPromptPart(content='give me the names of the first 10 presidents of the united states',
timestamp=datetime.datetime(2025, 11, 5, 16, 2, 26, 299586, tzinfo=datetime.timezone.utc))], instructions='Always 
respond with a JSON object that\'s compatible with this schema:\n\n{"$defs": {"PresidentialTerm": {"properties": 
{"name": {"type": "string"}, "term_start_year": {"type": "integer"}, "term_end_year": {"type": "integer"}}, 
"required": ["name", "term_start_year", "term_end_year"], "title": "PresidentialTerm", "type": "object"}}, 
"properties": {"presidents": {"items": {"$ref": "#/$defs/PresidentialTerm"}, "type": "array"}}, "required": 
["presidents"], "title": "Presidents", "type": "object", "description": "A list of presidential terms"}\n\nDon\'t 
include any text or Markdown fencing before or after.'),
    ModelResponse(parts=[ThinkingPart(content='We need to output JSON according to schema: object with "presidents"
array of items with properties name, term_start_year, term_end_year. Need first 10 US presidents: 1 George 
Washington 1789-1797, 2 John Adams 1797-1801, 3 Thomas Jefferson 1801-1809, 4 James Madison 1809-1817, 5 James 
Monroe 1817-1825, 6 John Quincy Adams 1825-1829, 7 Andrew Jackson 1829-1837, 8 Martin Van Buren 1837-1841, 9 
William Henry Harrison 1841-1841 (died), 10 John Tyler 1841-1845. Provide name, term_start_year, term_end_year. 
Note term_end_year of Harrison is 1841. Good. Output JSON only.', id='reasoning', provider_name='ollama'), 
TextPart(content='{"presidents":[{"name":"George 
Washington","term_start_year":1789,"term_end_year":1797},{"name":"John 
Adams","term_start_year":1797,"term_end_year":1801},{"name":"Thomas 
Jefferson","term_start_year":1801,"term_end_year":1809},{"name":"James 
Madison","term_start_year":1809,"term_end_year":1817},{"name":"James 
Monroe","term_start_year":1817,"term_end_year":1825},{"name":"John Quincy 
Adams","term_start_year":1825,"term_end_year":1829},{"name":"Andrew 
Jackson","term_start_year":1829,"term_end_year":1837},{"name":"Martin Van 
Buren","term_start_year":1837,"term_end_year":1841},{"name":"William Henry 
Harrison","term_start_year":1841,"term_end_year":1841},{"name":"John 
Tyler","term_start_year":1841,"term_end_year":1845}]}')], usage=RequestUsage(input_tokens=437, output_tokens=201), 
model_name='gpt-oss:20b', timestamp=datetime.datetime(2025, 11, 5, 16, 2, 38, tzinfo=TzInfo(0)), 
provider_name='ollama', provider_details={'finish_reason': 'stop'}, provider_response_id='chatcmpl-716', 
finish_reason='stop')
]

# Use to Create a Resume

In [25]:
with open("GEORGE_WASHINGTON_SHORT.md", "r", encoding="utf-8") as f:
    biography = f.read()

In [26]:
from pydantic import BaseModel
from dataclasses import dataclass
from pydantic_ai import Agent, RunContext, NativeOutput


@dataclass
class Biography:
    text: str


class PersonalExperience(BaseModel):
    title: str
    start_year: int
    end_year: int
    description: str
    major_accomplishments: list[str]
    skills: list[str]

class StructuredBio(BaseModel):
    experiences: list[PersonalExperience]
    education: list[str]
    skills: list[str]
    accomplishments: list[str]

agent = Agent[None, StructuredBio](
    model,
    deps_type=Biography,
    output_type=NativeOutput(StructuredBio),
)

@agent.instructions
async def instructions(ctx: RunContext[Biography]) -> str:
    return f"""
    You are a helpful assistant that can analyze a biography and extract structured
    information about the person's personal experiences, education, skills, and accomplishments.

    Here is the biography:
    {ctx.deps.text}
    """



In [27]:
result = await agent.run(deps=Biography(text=biography))

structured_bio = result.output
rich_print(structured_bio)
rich_print(result.usage())

StructuredBio(
    experiences=[
        PersonalExperience(
            title='Early childhood & family background',
            start_year=1732,
            end_year=1732,
            description='Born on 22\u202fFeb\u202f1732 in Westmoreland County, Virginia into a planter family, 
learning the values of discipline and responsibility.',
            major_accomplishments=[],
            skills=['adaptability', 'self‑discipline']
        ),
        PersonalExperience(
            title='Surveyor apprenticeship',
            start_year=1751,
            end_year=1753,
            description='Worked for Lord Fairfax, mapping the frontier; first formal technical training that would 
prove invaluable in military logistics.',
            major_accomplishments=[],
            skills=['spatial reasoning', 'data collection', 'reporting']
        ),
        PersonalExperience(
            title='Marriage to Martha Dandridge Custis',
            start_year=1759,
            end_year=1759,
            description='Union with the wealthy widow, strengthening social standing and securing the Mount Vernon 
estate as a planter’s hub.',
            major_accomplishments=[],
            skills=['networking', 'estate management']
        ),
        PersonalExperience(
            title='Post‑war retirement & plantation management',
            start_year=1758,
            end_year=1774,
            description='Returned to Mount Vernon; oversaw plantation operations, cultivated ties with local 
Virginia politics, and maintained an active seat in the House of Burgesses.',
            major_accomplishments=[],
            skills=['business administration', 'political lobbying']
        ),
        PersonalExperience(
            title='Post‑presidency retirement',
            start_year=1797,
            end_year=1799,
            description='Resigned after two presidential terms, withdrew to Mount Vernon, and died on 
14\u202fDec\u202f1799 after a brief illness.',
            major_accomplishments=[],
            skills=['retreat from power', 'legacy management']
        )
    ],
    education=[
        'Limited formal schooling; primary education likely provided by family.',
        'Self‑educated through books, military correspondence, and apprenticeship as a surveyor.',
        'Practical military training acquired during the French & Indian War (1754‑1763) and tenure in the 
Continental Army (1775‑1783).'
    ],
    skills=[
        'Strategic warfare & logistics – able to conduct guerilla and attritional campaigns against superior 
forces.',
        'Leadership – inspired loyalty, maintained troop morale across harsh winters and battles.',
        'Diplomacy & negotiation – mediated treaties (Jay, San\u202fLorenzo), negotiated with foreign powers and 
domestic factions.',
        'Political vision – articulated governance structures that balanced state and federal powers.',
        'Public speaking & precedent‑setting – crafted inaugural speeches, Farewell Address, and ceremonial 
protocols.',
        'Financial acumen – endorsed national bank, managed the federal mint, and supervised fiscal legislation.'
    ],
    accomplishments=[
        'Military victories: Siege of Boston (1776), Relief at Trenton (1776), Princeton (1777), decisive surrender
of Cornwallis at Yorktown (1781).',
        'Peaceful transition of power: voluntarily surrendered commission (1775), stepped down as President after 
two terms (1797).',
        'Presidential initiatives: signed Judiciary Act (1789), Naturalization Act (1790), Residence Act (1790 – 
Washington, D.C. established), Copyright Act (1790), Bank Act (1791), Mint creation, authorized first Navy 
frigates.',
        'Foreign policy: issued Proclamation of Neutrality (1793); negotiated Jay Treaty (1794) and Treaty of 
San\u202fLorenzo (1795).',
        'Constitutional contribution: presided over the 1787 Constitutional Convention and facilitated state 
ratification; elected as the first U.S. President (

RunUsage(input_tokens=2363, output_tokens=854, requests=1)

In [28]:
from pydantic_yaml import to_yaml_str


str_agent = Agent(model)

result = await str_agent.run(f"""
Convert the following information about George Washington into a resume suitable for a job application.
output in markdown format.

{to_yaml_str(structured_bio)}
""")

resume = result.output
rich_print(resume)

# George Washington  
> **Lead strategist, diplomat, and pioneer statesman**  
> Mount Vernon, VA — Washington, D.C.  
> Email: g.washington@patriots.org | Phone: (202) 555‑1234  

---

## Professional Summary  
Seasoned leader with 30 + years of military, governmental, and organizational experience. Proven record in 
directing large‑scale operations, negotiating complex international agreements, and establishing foundational 
governance structures. Adept at cultivating high‑performance teams, managing resources under pressure, and 
delivering results that set lasting precedents.

---

## Experience  

| **Title** | **Period** | **Key Responsibilities & Achievements** |
|-----------|------------|----------------------------------------|
| **President, United States of America** | 1789–1797 | • Delivered the inaugural address, launching the nation’s 
executive vision.  <br>• Authored and signed landmark federal legislation: Judiciary Act, Naturalization Act, 
Residence Act (establishing Washington, D.C.), Copyright Act, Bank Act.  <br>• Created the U.S. Mint, organized the
first Navy frigates, and established the first federal bank.  <br>• Negotiated the Jay Treaty (1794) and the Treaty
of San Lorenzo (1795) to secure peace and trade stability.  <br>• Issued the Proclamation of Neutrality to preserve
national security.  <br>• Resigned voluntarily after two terms, demonstrating a commitment to democratic 
transition. |
| **Commander, Continental Army** | 1775–1783 | • Led the Siege of Boston (1776), Relief at Trenton (1776), the 
Battle of Princeton (1777), and orchestrated the decisive surrender of Cornwallis at Yorktown (1781).  <br>• 
Directed logistics, supply lines, and troop morale in harsh winter campaigns.  <br>• Established disciplined, 
adaptive operations against a superior enemy force. |
| **Surveyor, Lord Fairfax’s Estate** | 1751–1753 | • Mapped frontier territories, performing spatial analysis, 
data collection, and reporting for strategic planning.  <br>• Developed accuracy‑driven survey techniques that 
later informed military logistics. |
| **Plantation Manager, Mount Vernon** | 1759–1774 | • Oversaw plantation operations, managing labor, resources, 
and supply chains.  <br>• Built and maintained political networks within Virginia’s House of Burgesses.  <br>• 
Integrated estate management with emerging colonial commerce. |
| **Colonial Politician, House of Burgesses** | 1774 | • Advocated for colonial interests, drafting legislation and
negotiating with British authorities. |
| **Early Life & Family Foundations** | 1732–1751 | • Cultivated discipline and responsibility from early 
adulthood; established self‑discipline and adaptability through plantation and surveying work. |

---

## Education  

| **Institution / Self‑Education** | **Period** | **Details** |
|---------------------------------|------------|-------------|
| **Family‑Run Primary School** | 1732‑1750 | Basic literacy, arithmetic, and civics. |
| **Self‑Education & Apprenticeship** | 1750‑1763 | Extensive reading of military correspondence, science books, 
and surveying manuals. |
| **Field Training – French & Indian War** | 1754‑1763 | Practical military training; gained experience in 
reconnaissance, logistics, and combat leadership. |
| **Continental Army Service** | 1775‑1783 | Advanced military instruction, strategic planning, and command 
responsibilities. |

---

## Core Competencies  

- **Strategic Warfare & Logistics** – Mastery of attrition and guerrilla tactics against larger forces.  
- **Leadership & Morale** – Built cohesive units; sustained confidence in extreme conditions.  
- **Diplomacy & Negotiation** – Successful treaty negotiations; balanced domestic & foreign agendas.  
- **Governance & Public Administration** – Designed federal structures, oversaw legislative enactments, and 
maintained executive accountability.  
- **Financial Stewardship** – Instituted national banking system, mint operations, and statutory fiscal policy.

In [29]:
#with open("GEORGE_WASHINGTON_RESUME.md", "w", encoding="utf-8") as f:
#    f.write(resume)